In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2'
sys.path.insert(0,my_path + r'/data')

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle

# Build features through Matrix Factorization

In [2]:
import os
os.chdir('D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data')

In [ ]:
os.system('build_vocab.sh')
os.system('cut_vocab.sh')

In [ ]:
from pickle_vocab import main as pickle_vocab
from cooc import main as cooc

pickle_vocab()
cooc()# cooccurrence matrix

In [ ]:
from glove_solution import main as glove

glove()

In [3]:
X = np.load('embeddings.npy')
print(X.shape)

(21161, 20)


In [4]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
    
print(len(vocab))
print(vocab)

21161
{'<user>': 0, '!': 1, 'i': 2, 'the': 3, '.': 4, ',': 5, 'to': 6, 'you': 7, '(': 8, '<url>': 9, 'a': 10, '...': 11, 'and': 12, 'my': 13, 'me': 14, 'of': 15, '?': 16, 'is': 17, 'for': 18, 'in': 19, 'it': 20, '"': 21, 'this': 22, 'so': 23, '-': 24, 'with': 25, 'on': 26, 'that': 27, ')': 28, 'be': 29, "i'm": 30, 'have': 31, ':': 32, 'but': 33, 'just': 34, 'rt': 35, 'love': 36, 'your': 37, 'all': 38, 'not': 39, 'was': 40, 'at': 41, 'are': 42, '..': 43, 'like': 44, '/': 45, 'get': 46, 'up': 47, 'frame': 48, '&': 49, 'lol': 50, 'know': 51, 'good': 52, 'do': 53, 'u': 54, 'now': 55, 'when': 56, 'one': 57, 'if': 58, 'we': 59, 'follow': 60, 'no': 61, 'can': 62, 'what': 63, 'go': 64, "don't": 65, 'out': 66, 'x': 67, 'will': 68, 'day': 69, "'": 70, 'please': 71, 'from': 72, 'see': 73, 'too': 74, 'want': 75, 'there': 76, 'back': 77, "it's": 78, 'today': 79, 'about': 80, 'really': 81, 'how': 82, 'got': 83, 'thanks': 84, '2': 85, "can't": 86, 'time': 87, 'its': 88, 'think': 89, 'im': 90, '*': 91

In [5]:
'):' in vocab

False

In [6]:
remove_list = ['<user>', '(', ')']

# build positive tweet feature set
X_pos = []

with open('train_pos.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_pos.append(list(np.mean(X_tweet, 0)))
        
    X_pos = np.array(X_pos)

In [7]:
# build negative tweet feature set
X_neg = []

with open('train_neg.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_neg.append(list(np.mean(X_tweet, 0)))
        
    X_neg = np.array(X_neg)     

In [8]:
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [9]:
print(X_pos.shape)
print(X_neg.shape)

(99947, 20)
(99988, 20)


# Training and testing sets

In [10]:
X_pos_neg = np.concatenate([X_pos, X_neg])
y_pos_neg = np.concatenate([y_pos, y_neg])

In [11]:
ind = np.arange(y_pos_neg.shape[0])
np.random.shuffle(ind)

r = 0.7
ir = round(r*X_pos_neg.shape[0])

X_tr = X_pos_neg[ind[:ir],:]
X_te = X_pos_neg[ind[ir:],:]
y_tr = y_pos_neg[ind[:ir]]
y_te = y_pos_neg[ind[ir:]]

# SVM 

In [12]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_tr, y_tr) 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Submission

In [ ]:
X_test = []

with open('test.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        tweet = ' '.join([word for word in tweet if word not in remove_list])

        X_tweet = []
        for w, word in enumerate(tweet.split()):

            if word in vocab:
                X_tweet.append(X[vocab.get(word),:])
                
        if X_tweet:
            X_neg.append(list(np.mean(X_tweet, 0)))
        
    X_neg = np.array(X_neg)     